<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/supervised_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import warnings

# Desactiva las advertencias innecesarias
warnings.filterwarnings('ignore')

# Configuración de pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Lee el archivo CSV y lo almacena en un DataFrame
try:
    df = pd.read_csv("test_labels.csv")
    # df = pd.read_csv("processed_dataset.csv")
except FileNotFoundError:
    print("Error: Archivo no encontrado. Asegúrate de que el archivo 'processed_dataset_to_train_models.csv' está en el directorio correcto.")
    exit()

# Muestra un resumen inicial del DataFrame
print("Primeras filas del DataFrame:")
print(df.head())

Primeras filas del DataFrame:
   Unnamed: 0                                                 ID  label
0           0  3333128835400580213056325007513731082455233755...      0
1           1  3333128835400580213056325007513731082455233755...      0
2           2  3333128835400580213056325007513731082455233755...      0
3           3  3333128835400580213056325007513731082455233755...      0
4           4  3333128835400580213056325007513731082455233755...      0


In [3]:
print("Columnas disponibles en el DataFrame:")
print(df.columns.tolist())

Columnas disponibles en el DataFrame:
['Unnamed: 0', 'ID', 'label']


In [5]:

# Columnas seleccionadas para SelectKBest
selectKBest_columns = [
    'Payment_6804', 'Base_80863', 'Infraction_QJJF', 'Base_76065',
    'Infraction_TLPJ', 'Base_1165', 'Base_39598', 'Base_85131', 'Base_9516',
    'Infraction_BSU', 'Infraction_ZYW', 'Infraction_TBP', 'Infraction_PBC',
    'Base_0229', 'Base_69608', 'Base_3041', 'Infraction_QKZN',
    'Infraction_CZE', 'Base_9103', 'Base_67254_low', 'label'
]

# Columnas seleccionadas para RFE
rfe_columns = [
    'Payment_6804', 'Base_80863', 'Expenditure_JIG', 'Base_02683',
    'Infraction_ZWWJ', 'Infraction_QJJF', 'Infraction_EJZ',
    'Infraction_GGO', 'Infraction_TLPJ', 'Base_1165', 'Base_39598',
    'Base_6187', 'Base_85131', 'Risk_9995', 'Infraction_AYWV', 'Base_9516',
    'Expenditure_HMO', 'Infraction_BSU', 'Infraction_ZYW', 'Infraction_TBP',
    'Infraction_PBC', 'Base_0229', 'Base_69608', 'Base_3041',
    'Infraction_QKZN', 'Infraction_CZE', 'Expenditure_MTRQ',
    'Infraction_XEPQ', 'Infraction_RKTA', 'Infraction_KEJT', 'label'
]

# Filtra el DataFrame para las columnas seleccionadas por SelectKBest
df_SelectKBest = df.loc[:, selectKBest_columns]
print("\nDataFrame con columnas SelectKBest:")
print(df_SelectKBest.head())

# Filtra el DataFrame para las columnas seleccionadas por RFE
df_rfe = df.loc[:, rfe_columns]
print("\nDataFrame con columnas RFE:")
print(df_rfe.head())

KeyError: "['Base_76065', 'Base_9103', 'Base_67254_low', 'label'] not in index"

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Función para entrenar y evaluar modelos
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test):
    """
    Entrena un modelo y evalúa su accuracy.

    Args:
        model: Instancia del modelo a entrenar.
        X_train: Conjunto de características para entrenamiento.
        X_test: Conjunto de características para prueba.
        y_train: Etiquetas para entrenamiento.
        y_test: Etiquetas para prueba.

    Returns:
        (str, float): Nombre del modelo y su precisión en los datos de prueba.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return model.__class__.__name__, acc

# Listas de DataFrames y nombres para iterar
dataframes = [
    (df_SelectKBest, 'SelectKBest'),
    (df_rfe, 'RFE')
]

# Modelos de clasificación a evaluar
models = [
    LogisticRegression(random_state=42, max_iter=1000),
    DecisionTreeClassifier(random_state=42),
    KNeighborsClassifier(),
    SVC(kernel='linear', random_state=42)
]

# Iterar sobre los DataFrames
for df, method_name in dataframes:
    print(f"\n=== Evaluación utilizando {method_name} ===")

    # Separar características (X) y variable objetivo (y)
    X = df.drop(columns=['label'])
    y = df['label']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    # Escalado de los datos
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Evaluar modelos y almacenar resultados
    results = [train_and_evaluate_model(model, X_train, X_test, y_train, y_test) for model in models]

    # Ordenar los resultados por accuracy (de mayor a menor)
    results_sorted = sorted(results, key=lambda x: x[1], reverse=True)

    # Mostrar resultados
    print("Resultados:")
    for model_name, acc in results_sorted:
        print(f"{model_name}: Accuracy = {acc:.4f}")

In [ ]:
import joblib

# Función para guardar objetos con joblib
def save_model_or_scaler(obj, filename, obj_type):
    """
    Guarda un objeto (modelo o scaler) en un archivo con joblib.

    Args:
        obj: Objeto a guardar (modelo o scaler).
        filename: Nombre del archivo donde se guardará el objeto.
        obj_type: Tipo de objeto ('scaler' o 'model') para mensajes informativos.

    Returns:
        None
    """
    try:
        joblib.dump(obj, filename)
        print(f"{obj_type.capitalize()} guardado exitosamente como '{filename}'")
    except Exception as e:
        print(f"Error al guardar el {obj_type}: {e}")

# Guardar el scaler
scaler_filename = "scaler.joblib"
save_model_or_scaler(scaler, scaler_filename, obj_type="scaler")

# Guardar el modelo k-NN
knn_model_filename = "knn_model.joblib"
knn_model = models[2][0]  # Suponiendo que models[2][0] es el modelo k-NN